In [1]:
from collections import defaultdict
import json
import os

ARCHIVE_DIR = './archive-Fall2019/'

# Notes

* Call numbers with less than 5 digits are meant to have leading zeroes, but I mistakenly turned them into int's in the scraper.

In [3]:
courses = []
dates = []

for d in range(0, 18):
    for f in reversed(sorted(os.listdir(ARCHIVE_DIR))):
        date = '2019-08-31' if d == 0 else f'2019-09-{d:02d}'
        if '.json'in f and date in f:
            dates.append(date)
            date_courses = json.load(open(ARCHIVE_DIR + f))
            for c in date_courses:
                c['date'] = date
                courses.append(c)
            # print(f'{date}: Added {len(date_courses)}.')
            break
print(f'Read in data from {len(dates)} dates.')

courses_indexed = defaultdict(dict)
for c in courses:
    courses_indexed[c['date']][c['Call Number']] = c
print('Indexed courses by date and call number.')

courses_lists = {}
for call_num in courses_indexed[dates[-1]].keys():
    if call_num not in courses_lists:
        courses_lists[call_num] = []
    for date in dates:
        courses_lists[call_num].append(
            courses_indexed[date].get(
                call_num,
                {'Call Number': call_num, 'date': date}
            )
        )
print('Made courses lists by call number.')

Read in data from 18 dates.
Indexed courses by date and call number.
Made courses lists by call number.


In [4]:
reassigned_calls = []
for call_num in courses_lists.keys():
    courses_instances = courses_lists[call_num]    
    if any(c.get('Location', None) == 'To be announced' for c in courses_instances):
        # At some point the course was To be announced    
        reassigned_calls.append(call_num)

        output = []
for call_num in reassigned_calls:
    output += courses_lists[call_num]
json.dump(output, open('./reassigned_calls.json', 'w'))

In [190]:
DATE_S = 13 # '2019-09-01'
DATE_E = 14 # '2019-09-02'

x = []
for call_num in courses_lists.keys():
    course_list = courses_lists[call_num]
    if course_list[DATE_E].get('Location', None) == 'To be announced':
        if course_list[DATE_S].get('Location', None) != course_list[DATE_E].get('Location', None):
            x.append(call_num)